<a href="https://colab.research.google.com/github/nsi319/Long-Document-Transformers/blob/main/led_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Clone finetune transformer repository**

In [12]:
# script to perform seq2seq training and evaluation
import os
from os import path
if path.exists("/content/drive/MyDrive/Finetune")==True:
  %rm -rf /content/drive/MyDrive/Finetune
  print("Deleted old repository")
!git clone https://github.com/nsi319/Finetune-Transformers.git /content/drive/MyDrive/Finetune

Deleted old repository
Cloning into '/content/drive/MyDrive/Finetune'...
remote: Enumerating objects: 835, done.
remote: Counting objects: 100% (835/835), done.
remote: Compressing objects: 100% (512/512), done.
remote: Total 835 (delta 332), reused 815 (delta 319), pack-reused 0
Receiving objects: 100% (835/835), 2.58 MiB | 3.47 MiB/s, done.
Resolving deltas: 100% (332/332), done.
Checking out files: 100% (379/379), done.


**Import libraries**

In [1]:
!pip install transformers==4.2.0
!pip install rouge_score
!pip install datasets
!pip install sentencepiece

     |████████████████████████████████| 1.8MB 11.9MB/s 
     |████████████████████████████████| 2.9MB 38.0MB/s 
     |████████████████████████████████| 890kB 40.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=90efa0cead0c03bf7603a56b94cceb872ca50205855e830a2a5294388eb1c9c9
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 184kB 11.2MB/s 
     |████████████████████████████████| 20.7MB 1.5MB/s 
     |████████████████████████████████| 102kB 10.7MB/s 
     |████████████████████████████████| 245kB 40.6MB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1
     |████████████████████████████████| 1.2MB 10.0MB/s 


**PREPARE TRAINING AND VALIDATION DATA**

In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


df = pd.read_csv("/content/drive/MyDrive/longformers/legal_dataset.csv")
df = df[["Text", "Summary"]]


length = [ [len(row["Summary"].split(" ")), len(row["Text"].split(" "))]  for index,row in df.iterrows()]

mean_text_length = int(np.mean(list(list(zip(*length))[1])))
mean_summ_length = int(np.mean(list(list(zip(*length))[0])))

max_text_length = int(np.max(list(list(zip(*length))[1])))
max_summ_length = int(np.max(list(list(zip(*length))[0])))

min_text_length = int(np.min(list(list(zip(*length))[1])))
min_summ_length = int(np.min(list(list(zip(*length))[0])))

print("Max Text Length: {} ".format(max_text_length))
print("Summary Length: [{},{}] ".format(min_summ_length,max_summ_length))
print("Mean Text Length: ",mean_text_length)
print("Mean Summary Length: ",mean_summ_length)


df_train, df_test = train_test_split(df, test_size=0.01)

df_train.to_csv("/content/drive/MyDrive/longformers/data/train.csv")
print(len(df_train))

df_test.to_csv("/content/drive/MyDrive/longformers/data/valid.csv")
print(len(df_test))


Max Text Length: 8466 
Summary Length: [142,833] 
Mean Text Length:  3850
Mean Summary Length:  352
100
2


In [ ]:
!nvidia-smi

Wed Feb 17 06:59:20 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# **Finetune LED**

**Using run.py (seq2seq finetuning) script from Finetune-Transformers**

In [ ]:
!python /content/drive/MyDrive/Finetune/run.py --help

2021-02-18 06:58:31.294966: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
usage: run.py [-h] --model_name_or_path MODEL_NAME_OR_PATH
              [--config_name CONFIG_NAME] [--tokenizer_name TOKENIZER_NAME]
              [--cache_dir CACHE_DIR] [--no_use_fast_tokenizer]
              [--model_revision MODEL_REVISION] [--use_auth_token]
              [--task TASK] [--dataset_name DATASET_NAME]
              [--dataset_config_name DATASET_CONFIG_NAME]
              [--text_column TEXT_COLUMN] [--summary_column SUMMARY_COLUMN]
              [--train_file TRAIN_FILE] [--validation_file VALIDATION_FILE]
              [--overwrite_cache]
              [--preprocessing_num_workers PREPROCESSING_NUM_WORKERS]
              [--max_source_length MAX_SOURCE_LENGTH]
              [--max_target_length MAX_TARGET_LENGTH]
              [--val_max_target_length VAL_MAX_TARGET_LENGTH]
              [--pad_to_max_length] [--max_tra

In [ ]:
!python /content/drive/MyDrive/Finetune/run.py  \
    --model_name_or_path allenai/led-base-16384 \
    --do_train \
    --do_eval \
    --task summarization \
    --max_source_length=5120 \
    --max_target_length=550 \
    --num_beams=3 \
    --min_summ_length=350 \
    --max_summ_length=500 \
    --length_penalty=2.0 \
    --train_file /content/drive/MyDrive/longformers/data/train.csv \
    --validation_file /content/drive/MyDrive/longformers/data/valid.csv \
    --output_dir /content/drive/MyDrive/longformers/models/legal-led-base-16384_max_st_6144_550_summ_350_500_split_86_16 \
    --overwrite_output_dir \
    --per_device_train_batch_size=1 \
    --per_device_eval_batch_size=1 \
    --predict_with_generate \
    --text_column Text \
    --summary_column Summary 

**Testing the trained model on complete dataset**

In [ ]:
!python /content/drive/MyDrive/Finetune/run.py  \
    --model_name_or_path /content/drive/MyDrive/longformers/models/legal-led-base-16384_max_st_6144_550_summ_350_500_split_86_16  \
    --do_eval \
    --task summarization \
    --max_source_length=6144 \
    --max_target_length=550 \
    --num_beams=3 \
    --min_summ_length=350 \
    --max_summ_length=500 \
    --length_penalty=2.0 \
    --validation_file /content/drive/MyDrive/longformers/legal_dataset.csv \
    --output_dir /content/drive/MyDrive/longformers/models/eval-legal-led-base-16384_max_st_6144_550_summ_350_500 \
    --overwrite_output_dir \
    --per_device_train_batch_size=1 \
    --per_device_eval_batch_size=1 \
    --predict_with_generate \
    --text_column Text \
    --summary_column Summary 

# **Test led-base-16384 with large text**

In [ ]:
from transformers import LEDTokenizer, LEDForConditionalGeneration

model = LEDForConditionalGeneration.from_pretrained("allenai/led-base-16384")
tokenizer = LEDTokenizer.from_pretrained("allenai/led-base-16384")

In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

padding = "max_length" 

text="""On March 2, 2018, the Securities and Exchange Commission announced securities fraud charges against a U.K.-based broker-dealer and its investment manager in connection with manipulative trading in the securities of HD View 360 Inc., a U.S.-based microcap issuer.  The SEC also announced charges against HD View's CEO, another individual, and three entities they control for manipulating HD View's securities as well as the securities of another microcap issuer, West Coast Ventures Group Corp.  The SEC further announced the institution of an order suspending trading in the securities of HD View.These charges arise in part from an undercover operation by the Federal Bureau of Investigation, which also resulted in related criminal prosecutions against these defendants by the Office of the United States Attorney for the Eastern District of New York.In a complaint filed in the U.S. District Court for the Eastern District of New York, the SEC alleges that Beaufort Securities Ltd. and Peter Kyriacou, an investment manager at Beaufort, manipulated the market for HD View's common stock.  The scheme involved an undercover FBI agent who described his business as manipulating U.S. stocks through pump-and-dump schemes.  Kyriacou and the agent discussed depositing large blocks of microcap stock in Beaufort accounts, driving up the price of the stock through promotions, manipulating the stock's price and volume through matched trades, and then selling the shares for a large profit.The SEC's complaint against Beaufort and Kyriacou alleges that they:opened brokerage accounts for the undercover agent in the names of nominees in order to conceal his identity and his connection to the anticipated trading activity in the accountssuggested that the undercover agent could create the false appearance that HD View's stock was liquid in advance of a pump-and-dump by "gam[ing] the market" through matched tradesexecuted multiple purchase orders of HD View shares with the understanding that Beaufort's client had arranged for an associate to simultaneously offer an equivalent number of shares at the same priceA second complaint filed by the SEC in the U.S. District Court for the Eastern District of New York alleges that in a series of recorded telephone conversations with the undercover agent, HD View CEO Dennis Mancino and William T. Hirschy agreed to manipulate HD View's common stock by using the agent's network of brokers to generate fraudulent retail demand for the stock in exchange for a kickback from the trading proceeds.  According to the complaint, the three men agreed that Mancino and Hirschy would manipulate HD View stock to a higher price before using the agent's brokers to liquidate their positions at an artificially inflated price.  The SEC's complaint also alleges that Mancino and Hirschy executed a "test trade" on Jan. 31, 2018, coordinated by the agent, consisting of a sell order placed by the defendants filled by an opposing purchase order placed by a broker into an account at Beaufort.  Unbeknownst to Mancino and Hirschy, the Beaufort account used for this trade was a nominal account that was opened and funded by the agent.  The SEC's complaint also alleges that, prior to their contact with the undercover agent, Mancino and Hirschy manipulated the market for HD View and for West Coast by using brokerage accounts that they owned, controlled, or were associated with –including TJM Investments Inc., DJK Investments 10 Inc., WT Consulting Group LLC – to effect manipulative "matched trades."The SEC's complaint against Beaufort and Kyriacou charges the defendants with violating Section 10(b) of the Securities Exchange Act of 1934 and Rule 10b-5 thereunder.  The SEC also charged Hirschy, Mancino, and their corporate entities with violating Section 17(a)(1) of the Securities Act of 1933, Sections 9(a)(1), 9(a)(2), and 10(b) of the Exchange Act and Rules 10b-5(a) and (c) thereunder.  The SEC is seeking injunctions, disgorgement, prejudgment interest, penalties, and penny stock bars from Beaufort and Kyriacou.  With respect to Hirschy, Mancino, and their corporate entities, the SEC is seeking injunctions, disgorgement, prejudgment interest, penalties, penny stock bars, and an officer-and-director bar against Mancino.The investigation was conducted in the SEC's New York Regional Office by Tejal Shah and Joseph Darragh, Lorraine Collazo, and Michael D. Paley of the Microcap Fraud Task Force and supervised by Lara S. Mehraban, and in Washington, D.C. by Patrick L. Feeney, Robert Nesbitt, and Kevin Guerrero, and supervised by Antonia Chion.  Preethi Krishnamurthy and Ms. Shah will lead the SEC's litigation against Beaufort and Kyriacou.  Ann H. Petalas and Mr. Feeney, under the supervision of Cheryl Crumpton, will handle the SEC's litigation against Mancino, Hirschy, and their entities.  The SEC appreciates the assistance of the Office of the United States Attorney for the Eastern District of New York, the Federal Bureau of Investigation, the Internal Revenue Service, the Alberta Securities Commission, the Ontario Securities Commission, the Financial Conduct Authority of the United Kingdom, and the Financial Industry Regulatory Authority.The Commission's investigation in this matter is continuing."""

# text= """Neural Transfer Learning with Transformers for Social Science Text Analysis Sandra Wankm¨uller Ludwig-Maximilians-University Munich sandra.wankmueller@gsi.lmu.de Abstract. During the last years, there have been substantial increases in the prediction performances of natural language processing models on text-based supervised learning tasks. Especially deep learning models that are based on the Transformer architecture (Vaswani et al., 2017) and are used in a transfer learning setting have contributed to this development. As Transformer-based models for transfer learning have the potential to achieve higher prediction accuracies with relatively few training data instances, they are likely to benefit social scientists that seek to have as accurate as possible text-based measures but only have limited resources for annotating training data. To enable social scientists to leverage these potential benefits for their research, this paper explains how these methods work, why they might be advantageous, and what their limitations are. Additionally, three Transformer-based models for transfer learning, BERT (Devlin et al., 2019), RoBERTa (Liu et al., 2019), and the Longformer (Beltagy et al., 2020), are compared to conventional machine learning algorithms on three social science applications. Across all evaluated tasks, textual styles, and training data set sizes, the conventional models are consistently outperformed by transfer learning with Transformer-based models, thereby demonstrating the potential benefits these models can bring to text-based social science research. Keywords. Natural Language Processing, Deep Learning, Neural Networks, Transfer Learning, Transformer, BERT 1 1 Introduction Deep learning architectures are well suited to capture the contextual and sequential nature of language and have enabled natural language processing (NLP) researchers to more accurately perform a wide range of tasks such as text classification, machine translation, or reading comprehension (Ruder, 2020). Despite the fact that deep learning techniques tend to exhibit higher prediction accuracies in text-based supervised learning tasks compared to traditional machine learning algorithms (Budhwar et al., 2018; Iyyer et al., 2014; Ruder, 2020), they are far from being a standard tool for social science researchers that use supervised learning for text analysis. Although there are exceptions (e.g. Amsalem et al., 2020; Chang and Masterson, 2020; Muchlinski et al., 2020; Rudkowsky et al., 2018; Wu, 2020; Zhang and Pan, 2019), for the implementation of supervised learning tasks social scientists typically resort to bagof-words vector space representations of texts that serve as an input to conventional machine learning models such as support vector machines (SVMs), naive Bayes, random forests, boosting algorithms, or regression with regularization (e.g. via the least absolute shrinkage and selection operator (LASSO)) (see e.g. Anastasopoulos and Bertelli, 2020; Barber´a et al., 2021; Ceron et al., 2015; Colleoni et al., 2014; Diermeier et al., 2011; D’Orazio et al., 2014; Fowler et al., 2020; Greene et al., 2019; Katagiri and Min, 2019; Kwon et al., 2018; Miller et al., 2020; Mitts, 2019; Park et al., 2020; Pilny et al., 2019; Ramey et al., 2019; Seb˝ok and Kacsuk, 2020; Theocharis et al., 2016; Welbers et al., 2017).1 One among several likely reasons why deep learning methods so far have not been widely used for supervised learning tasks by social scientists might be that training deep learning models is resource intensive. To estimate the exceedingly high number of parameters that deep learning models typically comprise, large amounts of training examples are required. For research questions relating to domains in which it is difficult to access or label large enough amounts of training data instances, deep learning becomes infeasible or prohibitively costly. Recent developments within NLP on transfer learning alleviate this problem. Transfer learning is a learning procedure in which representations learned on a source task are transmitted to improve learning on the target task (Ruder, 2019). In sequential transfer learning, commonly a language representation model, that comprises highly general, close to universal representations of language, is trained by conducting the source task (Ruder, 2019). Using these learned general-purpose representations as inputs to target 1This is not to say that social scientists would not have started to leverage the foundations of deep learning approaches in NLP: During the last years, the use of real-valued vector representations of terms, known as word embeddings, enabled social scientists to explore new research questions or to study old research questions by new means (Han et al., 2018; Kozlowski et al., 2019; Rheault et al., 2016; Rheault and Cochrane, 2020; Rodman, 2020; Watanabe, 2020). Yet, the analysis of social science relevant texts by means of deep learning up til now mostly is conducted by research teams that are not primarily social science trained (e.g. Abercrombie and Batista-Navarro, 2018; Ansari et al., 2020; Budhwar et al., 2018; Glavaˇs et al., 2017; Iyyer et al., 2014; Zarrella and Marsh, 2016). 2 tasks has been shown to improve the prediction performances on a wide range of these NLP target tasks (Ruder, 2020). Moreover, adapting the language representations to a target task requires fewer target training examples than when not using transfer learning and directly training the model on the target task (Howard and Ruder, 2018). Thus, research on transfer learning in NLP recently has made possible a new mode of learning in which ready-to-use language representation models, that are already pretrained on a source task, can be used off-the-shelf and be easily adapted with few resources to any target task of interest (Alammar, 2018a). In addition to the efficiency and performance gains from research on transfer learning, the introduction of the attention mechanism (Bahdanau et al., 2015), which became incorporated in the Transformer architecture presented by Vaswani et al. in 2017, has significantly improved the ability of deep learning NLP models to capture contextual information from texts. During the last years, several Transformer-based models that are used in a transfer learning setting have been introduced (e.g. Devlin et al., 2019; Liu et al., 2019; Yang et al., 2019). These models substantively outperform previous state-of-the-art models across a large variety of NLP tasks (Ruder, 2020). Due to the likely increases in prediction accuracy as well as the efficient and less resourceful adaptation phase, transfer learning with deep (e.g. Transformer-based) language representation models seems promising to social science researchers that seek to have as accurate as possible text-based measures but may lack the resources to annotate large amounts of data or are interested in specific domains in which only small corpora and few training instances exist. In order to equip social scientists to use the potentials of transfer learning with Transformer-based models for their research, this paper provides a thorough but focused introduction to transfer learning and the Transformer. The next two sections compare conventional to deep learning approaches and introduce basic concepts and methods of deep learning. These two sections lay the groundwork for the two following sections that present transfer learning and the Transformer architecture. The section on transfer learning provides an answer to the question what transfer learning is and explains in more detail in what ways transfer learning might be beneficial. The subsequent section then introduces the attention mechanism and the Transformer and elaborates on how the Transformer has advanced the study of text. Then, an overview over Transformer-based models for transfer learning is provided. Here a special focus will be given to the seminal Transformer-based language representation model BERT (standing for Bidirectional Encoder Representations from Transformers) (Devlin et al., 2019). Afterward, guidance on deep learning and transfer learning in practice is offered. Finally, three Transformer-based models for transfer learning, BERT (Devlin et al., 2019), RoBERTa (Liu et al., 2019), and the Longformer (Beltagy et al., 2020), are compared to traditional learning algorithms based on three classification tasks using data from speeches in the UK parliament (Duthie and Budzynska, 2018), tweets regarding the legalization of abortion (Mohammad et al., 2017), and comments from Wikipedia Talk pages (Jigsaw/Conversation AI, 2018). The final section concludes with a discussion on task-specific factors and research goals for which neural transfer learning 3 with Transformers is highly beneficial vs. rather limited. 2 Comparing Conventional Machine Learning to Deep Learning Starting with core machine learning concepts, this section works out the differences that conventional machine learning models vs. deep learning models imply for text analysis. Together with the next section, this section serves as the basis for the subsequently provided introductions to transfer learning and Transformer-based models. 2.1 Machine Learning Given raw input data X = {d1, . . . , dn, . . . , dN } (e.g. a corpus comprising N raw text files) and a corresponding output variable y = {y1, . . . , yn, . . . , yN }, the aim in supervised machine learning is to search the space of possible mappings between X and y to find the parameters θ of a function such that the estimated function’s predictions, yˆ = f(X , θˆ), are maximally close to the true values y and thus—given new, yet unseen data Xtest—the trained model will generate accurate predictions (Chollet, 2020). The distance between y and f(X , θˆ) is measured by a loss function, L(y, f(X , θˆ)), that guides the search process (Chollet, 2020). Learning in supervised machine learning hence means finding f(X , θˆ) and essentially is a two-step process (Goodfellow et al., 2016): The first step is to learn representations of the data, fR(X , θˆR), and the second is to learn mappings from these representations of the data to the output. yˆ = f(X , θˆ) = fO(fR(X , θˆR), θˆO) (1) 2.2 Conventional Machine Learning Conventional machine learning algorithms cover the second step: They learn a function mapping data representations to the output. This in turn implies that the first step falls to the researcher who has to (manually) generate representations of the data herself. When working with texts, the raw data X typically are a corpus of text documents. A very common approach then is to transform the raw text files via multiple preprocessing procedures into a document-feature matrix X = {x1, . . . , xn, . . . , xN } > (see Figure 1). In a document-feature matrix each document is represented as a feature vector xn = {xn1, . . . , xnk, . . . , xnK} (Turney and Pantel, 2010). Element xnk in this vector gives the value of document n on feature k—which typically is the (weighted) number of times that the kth textual feature occurs in the nth document (Turney and Pantel, 2010). To conduct the second learning step, the researcher then commonly applies a conventional machine learning algorithm on the document-feature matrix to learn the 4 I agree :-) I confess this is a problem. These people need help – now. Come on. This is nonsense. … I agree :-) confess this … 1 1 1 0 0 … 1 0 0 1 1 … 0 0 0 0 0 … 0 0 0 0 1 … … … … … … … 1 0 1 0 … 0 1 2 raw data representations of the data output e.g. corpus e.g. document-feature matrix Figure 1: Learning as a Two-Step Process. Learning in machine learning essentially is a two-step process. In text-based applications of conventional machine learning approaches, the raw data X first are (manually) preprocessed such that each example is represented as a feature vector in the document-feature matrix X. Second, these feature representations of the data are fed as inputs to a traditional machine learning algorithm that learns a mapping between data representations X and outputs y. relation between the document-feature representation of the data X and the provided response values y. There are two difficulties with this approach. The first is that it may be hard for the researcher to a priori know which features are useful for the task at hand (Goodfellow et al., 2016). The performance of a supervised learning algorithm will depend on the representation of the data in the document-feature matrix (Goodfellow et al., 2016). In a classification task, features that capture observed linguistic variation that helps in assigning the texts into the correct categories are more informative and will lead to a better classification performance than features that capture variation that is not helpful in distinguishing between the classes (Goodfellow et al., 2016). Yet, determining which sets of possibly highly abstract and complex features are informative and which are not is highly difficult (Goodfellow et al., 2016): A researcher can choose from a multitude of possible preprocessing steps such as stemming, lowercasing, removing stopwords, adding part-of-speech (POS) tags, or applying a sentiment lexicon.2 The set of selected preprocessing steps as well as the order in which they are implemented define the way in which the texts at hand are represented and thus affect the research findings (Denny and Spirling, 2018; Goodfellow et al., 2016). Social scientists may be able to use some of their domain knowledge in deciding upon a few specific preprocessing decisions (e.g. whether it is likely that excluding a predefined list of stopwords will be beneficial because it re2For a more detailed list of possible steps see Turney and Pantel (2010) and Denny and Spirling (2018). 5 duces dimensionality or will harm performance because the stopword list includes some terms that are important). Domain knowledge, however, is most unlikely to guide researchers regarding all possible permutations of preprocessing steps. Simply trying out each possible preprocessing permutation in order to select the best performing one for a supervised task is not possible given the massive number of permutations and limited researcher resources. The second problem is that in a document-feature matrix each document is represented as a single vector xn—which implies that each document is treated as a bag-of-words (Turney and Pantel, 2010). Bag-of-words representations disregard word order and syntactic or semantic dependencies between words in a sequence (Turney and Pantel, 2010).3 Yet, text is contextual and sequential by nature. Word order carries meaning. And the context in which a word is embedded in is essential in determining the meaning of a word. When represented as a bag-of-words, the sentence ‘The opposition party leader attacked the prime minister.’ cannot be distinguished from the sentence ‘The prime minister attacked the opposition party leader.’. Moreover, the fact that the word ‘party’ here refers to a political party rather than a festive social gathering only becomes clear from the context. Although bag-of-words models perform relatively well given the simple representations of text they build upon (Grimmer and Stewart, 2013), it has been shown that capturing contextual and sequential information is likely to enhance prediction performances (see for example Socher et al., 2013). 2.3 Deep Learning In contrast to conventional machine learning algorithms, deep learning models conduct both learning steps: they learn representations of the data and a function mapping data representations to the output. Deep learning is one branch of representation learning in which features are not designed and selected manually but are learned by a machine learning algorithm (Goodfellow et al., 2016). In deep learning models, an abstract representation of the data is learned by applying the data to a stack of several simple functions (Goodfellow et al., 2016). Each function takes as an input the representation of the data created by (the sequence of) previous functions and generates a new representation: f(X , θˆ) = fO(. . . fR3 (fR2 (fR1 (X , θˆR1 ), θˆR2 ), θˆR3 ). . . , θˆO) (2) The first function fR1 takes as an input the data and provides as an output a first representation of the data. The second function fR2 takes as an input the representation learned by the first function and generates a new representation and so on. Deep learning 3By counting the occurrence of word sequences of length N, N-gram models extend unigram-based bag-of-words models and allow for capturing information from small contexts around words. Yet, by including N-grams as features the dimensionality of the feature space increases, thereby increasing the problem of high dimensionality and sparsity. Moreover, texts often exhibit dependencies between words that are positioned much farther apart than what could be captured with N-gram models (Chang and Masterson, 2020). 6 models thus are characterized by providing a layered representation of the data in which each layer of representation is based on previous data representations (Goodfellow et al., 2016). Hereby, complex and abstract representations are learned from less abstract, more simple ones (Goodfellow et al., 2016). As they are composed of nested layers of functions, deep learning models are called deep (Goodfellow et al., 2016). The representation layers, {R1, R2, R3, . . . }, are named hidden layers (Goodfellow et al., 2016). The dimensionality of the vector-valued hidden layer outputs is a model’s width and the number of hidden layers is the depth of a model (Goodfellow et al., 2016). Deep learning models differ in their architecture. Most types of models, however, are based on a chain of functions as specified in equation 2 (Goodfellow et al., 2016). Deep learning models do learn representations of the data. Yet, when applying them to text-based applications they do not take as an input the raw text documents. They still have to be fed with a data format they can read. Rather than taking as an input a raw text document dn = {a1, . . . , at , . . . , aT }, which is a sequence of words (or more precisely: a sequence of tokens), deep learning models take as an input a sequence of word embeddings {z[a1] , . . . , z[at] , . . . , z[aT ]}. Word embeddings are real-valued vector representations of the unique terms in the vocabulary (Mikolov et al., 2013c). The vocabulary Z consists of the U unique terms in the corpus: Z = {z1, . . . , zu, . . . , zU }. Each vocabulary term zu can be represented as a word embedding—a K-dimensional real-valued vector zu ∈ R K. And as each token at is an instance of one of the unique terms in the vocabulary, each token has a corresponding embedding. A document {a1, . . . , at , . . . , aT } consequently can be mapped to {z[a1] , . . . , z[at] , . . . , z[aT ]}. A deep learning model thus is fed with a sequence of real-valued vectors instead of a sequence of tokens. The values of the word embedding vectors are parameters that are learned in the optimization process. Hence, the representation for each unique textual token is learned when training the model. Except for the decision of how to separate a continuous text sequence into tokens (and sometimes also whether to lowercase the tokens and consequently to only learn embeddings for lowercased tokens), deep learning models learn representations of the data rather than operating on manually tailored representations. Also note that deep learning NLP models—instead of learning one vector representation for each term—learn one vector per term in each hidden layer. As deep learning NLP models learn to represent a raw sequence of tokens as a sequence of vectors instead of taking it as a bag-of-words, deep learning models can learn dependencies between tokens and take into account contextual information. Deep learning architectures as recurrent neural networks (RNNs) (Elman, 1990) and the Transformer (Vaswani et al., 2017) are especially suited to capture dependencies between sequential input data and are able to encode information from the textual context a token is embedded in. 7 2.4 Well-Performing and Efficient Models for NLP Tasks The no free lunch theorem (Wolpert and Macready, 1997) states that if averaging over all possible classification tasks every classification algorithm will achieve the same generalization performance (Goodfellow et al., 2016). This theorem implies that there is no universally superior machine learning algorithm (Goodfellow et al., 2016). A specific algorithm will perform well on one task but less well on another. Hence, given a type of learning task, researchers ideally should develop and use machine learning algorithms that are especially suited to conduct this type of task. This is, researchers should employ algorithms that are good at approximating the functions that map from feature inputs to provided outputs in the real-world applications they encounter (Goodfellow et al., 2016). In NLP tasks, models have to learn mappings between textual inputs and task-specific outputs. Common NLP tasks are binary or multi-class classification tasks in which the model’s task is to assign one out of two or one out of several class labels to each text sequence. Other NLP tasks, for example, require the model to answer multiple choice questions about provided input texts or to make predictions about the entailment of a sentence pair (Wang et al., 2019b). Due to their layered architecture and the vector-valued representations deep learning models tend to have a high capacity. This is, they can approximate a large variety of complex functions (Goodfellow et al., 2016). On less complex data structures, large deep learning models may risk overfitting and conventional machine learning approaches with lower expressivity may be more suitable. The ability to express complicated functions as well as the ability to automatically learn representations and the ability to encode information on connections between sequential inputs, however, seems essential when working with textual data: Empirically, deep learning models outperform classic machine learning algorithms on nearly all NLP tasks (Ruder, 2020). Despite these advantages, deep learning methods so far are not widely used for supervised learning tasks within the quantitative text analysis community in social science. Some studies do use deep learning models (e.g. Amsalem et al., 2020; Chang and Masterson, 2020; Muchlinski et al., 2020; Rudkowsky et al., 2018; Wu, 2020; Zhang and Pan, 2019). Yet, until now social scientists typically approach supervised classification tasks by generating bag-of-words based representations of texts that then are passed on to conventional machine learning algorithms as SVMs, naive Bayes, regularized regression, or tree-based methods (see e.g. Anastasopoulos and Bertelli, 2020; Barber´a et al., 2021; Ceron et al., 2015; Colleoni et al., 2014; Diermeier et al., 2011; D’Orazio et al., 2014; Fowler et al., 2020; Greene et al., 2019; Katagiri and Min, 2019; Kwon et al., 2018; Miller et al., 2020; Mitts, 2019; Park et al., 2020; Pilny et al., 2019; Ramey et al., 2019; Seb˝ok and Kacsuk, 2020; Theocharis et al., 2016; Welbers et al., 2017). There are several factors that might have contributed to the sporadic rather than widespread use of deep learning models within text-based social science. One likely 8 reason is that deep learning models have considerably more parameters to be learned in training than classic machine learning models. Consequently, deep learning models are computationally highly intensive and require substantially larger amounts of training examples. How much more training data are needed depends on the width and depth of the model, the task, and training data quality. Thus, precise numbers on the amounts of parameters and required training examples cannot be specified. To nevertheless put the sizes in relation note that a SVM with a linear kernel that learns to construct a hyperplane in a 3000-dimensional feature space which separates instances into two categories based on 1000 support vectors has around 3 million parameters. The Transformer-based models presented in this article, in contrast, have well above 100 million parameters. Recent developments of deep language representation models for transfer learning, however, reduce the amount of training instances needed to achieve the same level of performance than when not using transfer learning (Howard and Ruder, 2018). The introduction of the Transformer (Vaswani et al., 2017) additionally has improved the study of text. Due to its self-attention mechanisms the Transformer is better able to encode contextual information and dependencies between tokens than previous deep learning architectures (Vaswani et al., 2017). To enable social science research to leverage these potentials, this study presents and explains the workings and usage of Transformer models for transfer learning. 3 Introduction to Deep Learning This section provides an introduction to the basics of deep learning and thereby lays the foundation for the following sections. First, based on the example of feedforward neural networks (FNNs) the core elements of neural network architectures are explicated. Then, the optimization process via stochastic gradient descent with backpropagation (Rumelhart et al., 1986) will be presented. Subsequently, the architecture of recurrent neural networks (RNNs) (Elman, 1990) is outlined. 3.1 Feedforward Neural Network The most elementary deep learning model is a feedforward neural network (FNN), also named multilayer perceptron (Goodfellow et al., 2016). A feedforward neural network with L hidden layers, vector input x and a scalar output y can be visualized as in Figure 2 and be described as follows (Ruder, 2019, see also): h1 = σ1(W1x + b1) (3) h2 = σ2(W2h1 + b2) (4) . . . hl = σl(Wlhl−1 + bl) (5) 9 . . . y = σO(wOhL + bO) (6) The input to the neural network is the K0-dimensional vector x (see equation 3). x enters an affine function characterized by weight matrix W1 and bias vector b1, whereby W1 ∈ R K1×K0 , and b1 ∈ R K1 . σ1 is a nonlinear activation function and h1 ∈ R K1 is the K1-dimensional representation of the data in the first hidden layer. This is, the neural networks takes the input data x and via combining an affine function with a nonlinear activation function generates a new, transformed representation of the original input: h1. The hidden state h1 in turn serves as the input for the next layer that produces representation h2 ∈ R K2 . This continues through the layers until the last hidden representation, hL ∈ R KL , enters the output layer (see equation 6). � �" �. �" �# �. �" �# �$ ℎ.. ℎ." ℎ.# ℎ.$ ℎ%. ℎ%" ℎ%# ℎ%$ �$ ℎ&. ℎ&" ℎ&# ℎ&$ � Figure 2: Feedforward Neural Network. Feedforward Neural Network with L hidden layers, four units per hidden layer and scalar output y. The solid lines indicate the linear transformations encoded in weight matrix W1. The dotted lines indicate the connections between several consecutive hidden layers. The activation functions in neural networks typically are chosen to be nonlinear (Goodfellow et al., 2016). The reason is that if the activation functions were set to be linear, the output of the neural network would merely be a linear function of x (Goodfellow et al., 2016). Hence, the use of nonlinear activation functions is essential for the capacity of neural networks to approximate a wide range of functions and highly complex functions (Ruder, 2019). In the hidden layers, the rectified linear unit (ReLU) (Nair and Hinton, 2010) is often used as an activation function σl (Goodfellow et al., 2016). If q = {q1, . . . , qk, . . . , qK} is the K-dimensional vector resulting from the affine transformation in the lth hidden layer, q = Wlhl−1 +bl (see equation 5), then ReLU is applied on each element qk: σl(q)k = max{0, qk} (7) 10 σl(q)k then is the kth element of hidden state vector hl . 4 In the output layer, the activation function σO is selected so as to produce an output that matches the task-specific type of output values. In binary classification tasks with yn ∈ {0, 1} the standard logistic function, often simply referred to as the sigmoid function, is a common choice (Goodfellow et al., 2016). For a single observational unit n, the sigmoid function’s scalar output value is interpreted as the probability that yn = 1. If yn, however, can assume one out of C unordered response category values, yn ∈ {1, . . . , c, . . . , C}, the softmax function, a generalization of the sigmoid function that takes as an input and produces as an output a vector of length C, is typically employed (Goodfellow et al., 2016). For the nth example, the cth element of the softmax output vector gives the probability predicted by the model that unit n falls into class c. 3.2 Optimization: Gradient Descent with Backpropagation In supervised learning tasks, a neural network is provided with input xn and corresponding output yn for each training example. All the weights and bias terms are parameters to be learned in the process of optimization (Goodfellow et al., 2016). The set of parameters hence is θ = {W1, . . . ,Wl , . . . ,WL,WO, b1, . . . , bl , . . . , bL, bO}. For a single training example n, the loss function L(yn, f(xn, θˆ)) measures how well the value predicted for n by model f(xn, θˆ), that is characterized by the estimated parameter set θˆ, matches the true value yn. In the optimization process the aim is to find the set of values for the weights and biases that minimizes the average of the observed losses over all training set instances, also known as the empirical risk: R(θˆ) = 1 N PN n=1 L(yn, f(xn, θˆ)) (Goodfellow et al., 2016). Neural networks commonly employ variants of gradient descent with backpropagation in the optimization process (Goodfellow et al., 2016). To approach the local minimum of the empirical risk function, the gradient descent algorithm makes use of the fact that the direction of the negative gradient of function R at current point θˆi gives the direction in which R is decreasing fastest—the direction of the steepest descent (Goodfellow et al., 2016). The gradient is a vector of partial derivatives. It is the derivative of R at point θˆi and is commonly denoted as ∇θˆi R(θˆi). So, in the ith iteration, the gradient descent algorithm computes the negative gradient of R at current point θˆi and then moves from θˆi into the direction of the negative gradient: θˆi+1 = θˆi − η∇θˆi R(θˆi) (8) whereby η ∈ R+ is the learning rate. If η is small enough, then R(θˆi) ≥ R(θˆi+1) ≥ R(θˆi+2) ≥ . . . . This is, repeatedly computing the gradient and updating into the 4Activation functions that are similar to ReLU are the Exponential Linear Unit (ELU) (Clevert et al., 2016), Leaky ReLU and the Gaussian Error Linear Unit (GELU) (Hendrycks and Gimpel, 2016). The latter is used in BERT (Devlin et al., 2019). 11 direction of the negative gradient with small enough learning rate η, will generate a sequence moving toward the local minimum (Li et al., 2020a). In each iteration, the gradients for all parameters are computed via the backpropagation algorithm (Rumelhart et al., 1986).5 A very frequently employed approach, known as mini-batch gradient descent or stochastic gradient descent, is to compute the gradients based on a small random sample, a mini-batch, of M training set observations: ∇θˆi R(θˆi) = 1 M X M m=1 ∇θˆi L(ym, f(xm, θˆi)) (9) The gradient computed on a sample of training instances usually provides a good approximation of the loss function gradient evaluated on the entire training set (Goodfellow et al., 2016). It requires fewer computational resources and thus leads to faster convergence (Goodfellow et al., 2016). M typically assumes a value in the range from 2 to a few hundred (Ruder, 2019). The size of the mini-batch M and the learning rate η are hyperparameters in training neural network models. Especially the learning rate is often carefully tuned (Li et al., 2020a). A too high learning rate leads to large oscillations in the loss function values, whereas a too low learning rate implies slow convergence and risks getting stuck at a nonoptimal point with a high loss value (Goodfellow et al., 2016). Commonly, the learning rate is not kept constant but is set to decrease over time (Goodfellow et al., 2016). Furthermore, there are variants of stochastic gradient descent, as AdaGrad (Duchi et al., 2011), RMSProp (Hinton et al., 2012), and Adam (Kingma and Ba, 2015), that have a varying learning rate for each individual parameter (Goodfellow et al., 2016). 3.3 Recurrent Neural Networks The recurrent neural network (RNN) (Elman, 1990) is the most basic neural network to process sequential input data of variable length such as texts (Goodfellow et al., 2016). Given an input sequence of T token embeddings {z[a1] , . . . , z[at] , . . . , z[aT ]}, RNNs sequentially process each token. Here, one input token embedding z[at] corresponds to one time step t and the hidden state ht is updated at each time step. At each step t, the hidden state ht is a function of the hidden state generated in the previous time step, ht−1, as well as new input data, z[at] (see Figure 3) (Amidi and Amidi, 2019; Elman, 1990). The hidden states ht , that are passed on and transformed through time, serve as the model’s memory (Elman, 1990; Ruder, 2019). They capture the information of the sequence that entered until t (Goodfellow et al., 2016). Due to this sequential architecture, RNNs theoretically can model dependencies over the entire range of an input sequence 5The backpropagation algorithm makes use of the chain rule to compute the gradients. Helpful introductions to the backpropagation algorithm can be found in Li et al. (2020a), Li et al. (2020b) and Hansen (2020). 12 ℎ. ℎ" ℎ#$. ". "" "#$. … "# … ℎ# #[&.] #[&"] #[&#$.] #[&#] Figure 3: Recurrent Neural Network. Architecture of a basic recurrent neural network unfolded through time. At time step t, the hidden state ht is a function of the previous hidden state, ht−1, and current input embedding z[at] . yt is the output produced at t. (Amidi and Amidi, 2019). Yet, practically recurrent models have problems in learning dependencies extending beyond sequences of 10 or 20 tokens (Goodfellow et al., 2016). The reason is that when backpropagating the gradients through the time steps (this is known as Backpropagation Through Time (BPTT)), the gradients may vanish and thus fail to transmit a signal over long ranges (Goodfellow et al., 2016). The Long Short-Term Memory (LSTM) model (Hochreiter and Schmidhuber, 1997) extends the RNN with input, output, and forget gates that enable the model to accumulate, remember as well as forget provided information (Goodfellow et al., 2016). This makes them better suited than the basic RNNs to model dependencies stretching over long time spans as found in textual data (Ruder, 2019). 4 Transfer Learning Transformer-based models for transfer learning fuse two lines of recent NLP research: transfer learning and attention. Both concepts and respective developments will be outlined in the following two sections such that after reading these sections the reader will not only be prepared to understand the workings of transfer learning with Transformerbased models like BERT but also will be informed about latest major developments within NLP. This first section defines transfer learning and describes the benefits and uses of transfer learning. 4.1 A Taxonomy of Transfer Learning The classic approach in supervised learning is to have a training data set containing a large number of annotated instances, {xn, yn} N n=1, that are provided to a model that learns a function relating the xn to the yn (Ruder, 2019). If the train and test data instances have been drawn from the same distribution over the feature space, the trained 13 model can be expected to make accurate predictions for the test data, i.e. to generalize well (Ruder, 2019). Given another task (e.g. another set of labels to learn) or another domain (e.g. another set of documents with a different thematic focus), the standard supervised learning procedure would be to sample and create a new training data set for this new task and domain (Pan and Yang, 2010; Ruder, 2019). This is, for each new task and domain a new model is trained from the start (Pan and Yang, 2010; Ruder, 2019). There is no transferal of already existing, potentially relevant and useful information from related domains or tasks to the task at hand (Ruder, 2019). Trained supervised learning models thus are not good at generalizing to data exhibiting characteristics different from the data they have been trained on (Ruder, 2019). Moreover, the (manual) labeling of thousands to millions of training instances for each new task makes supervised learning highly resource intensive and prohibitively costly to be applied for all potentially useful and interesting tasks (Pan and Yang, 2010; Ruder, 2019). In situations in which the number of annotated training examples is restricted or the researcher lacks the resources to label a sufficiently large number of training instances classic supervised learning fails (Ruder, 2019). This is where transfer learning comes in. Transfer learning refers to statistical learning procedures in which knowledge learned by training a model on a related task and domain, the source task and source domain, is transferred to the learning process of the target task in the target domain (Pan and Yang, 2010; Ruder, 2019). Ruder (2019) provides a taxonomy of transfer learning scenarios in NLP. In transductive transfer learning, the source and target tasks are the same but annotated training examples only are available for the source domain (Ruder, 2019). Here, knowledge is transferred across domains (domain adaptation) or languages (cross-lingual learning) (Ruder, 2019). In inductive transfer learning source and target tasks differ but the researcher has access to at least some labelled training samples in the target domain (Ruder, 2019). In this setting, tasks can be learned simultaneously (multi-task learning) or sequentially (sequential transfer learning) (Ruder, 2019). 4.2 Sequential Transfer Learning In this article the focus is on sequential transfer learning, which at present is the most frequently employed type of transfer learning (Ruder, 2019). In sequential transfer learning the source task differs from the target task and training is conducted in a sequential manner (Ruder, 2019). Here, two stages are distinguished (see also Figure 4): First, a model is trained on the source task (pretraining phase) (Ruder, 2019). Subsequently, the knowledge gained in the pretraining phase is transmitted to the model trained on the target task (adaptation phase) (Ruder, 2019). In NLP, the knowledge that is transferred are the parameter values learned during training the source model—and thus also includes the values of the token representation vectors. The source model thus by some authors is also referred to as a language representation model (see e.g. Devlin et al., 2019). 14 $."#$%& $'($)&' %&'(&)*+*+, .'($)&' , #'($)&' $'($)&' )-)%()(*.+ We laughed out loud [EOS] He is eating an apple [EOS] She told me to come [EOS] She nodded [EOS] New York woke up [EOS] The sun shone bright ##ly [EOS] .."#$%& , #."#$%& Figure 4: Pretraining and Adaptation Phase in Sequential Transfer Learning. In the pretraining phase of sequential transfer learning a model is trained on the source task {Xsource, ysource}. The representations (i.e. the parameters) learned in the pretraining phase then are taken to the learning process of the target task, {Xtarget, ytarget}. A once pretrained model can serve as an input for a large variety of target tasks. 15 The common procedure in sequential transfer learning in NLP is to select a source task that will learn very general, close to universal representations of language in the pretraining phase that then provide an effective, well-generalizing input for a large spectrum of specific target tasks (Ruder, 2019). As many training instances are required to learn such general language representations, training a source model in the sequential transfer learning setting is highly expensive (Ruder, 2019). Yet, adapting a once pretrained model to a target task is often fast and cheap as transfer learning procedures require only a small proportion of the annotated target data required by standard supervised learning procedures to achieve the same level of performance (Howard and Ruder, 2018). In Howard and Ruder (2018), for example, training the deep learning model ULMFiT (short for Universal Language Model Fine-Tuning) from scratch on the target task requires 5 to 20 times more labeled training examples to reach the same error rate than using ULMFiT for transfer learning. Beside being more efficient, transfer learning also tends to increase prediction performances: Starting with the work by Yosinski et al. (2014) and Donahue et al. (2014), the computer vision community discovered that when the weights that had been learned by training deep learning models on the ImageNet Dataset (a data set containing annotations for more than 14 million images (Deng et al., 2009)) are used as pretrained inputs to other computer vision target tasks, this substantially increases the prediction performances on the target tasks—even if only few target training instances are used (Ruder, 2019). In computer vision, sequential transfer learning with pretraining on the ImageNet Dataset has proven so effective that it has been established as a widely used learning procedure for several years now (Ruder, 2019). Also in NLP, transfer learning based on models as ULMFiT (Howard and Ruder, 2018), OpenAI GPT (standing for Generative Pretrained Transformer) (Brown et al., 2020; Radford et al., 2018, 2019), BERT (Devlin et al., 2019), and XLNet (Yang et al., 2019) have led to substantial increases in prediction performances in recent years (Ruder, 2019). The smaller the target task training data set size, the more salient the pretrained representations become. When decreasing the number of target task training set instances, the prediction performance of deep learning models that are trained from scratch on the target task declines (Howard and Ruder, 2018). For deep neural networks that are used in a transfer learning setting and receive well-generalizing representations from pretraining, in contrast, prediction performance levels on the target task decrease more slowly and slightly with diminishing numbers of training set instances (Howard and Ruder, 2018). So, transfer learning is likely to enhance prediction performances on a target task. These enhancements are likely to be especially strong for medium-sized and small target task training data sets (Howard and Ruder, 2018). 4.3 Pretraining The discoveries with the ImageNet Dataset suggest that in order to learn general, close to universal representations, that are relevant for a wide range of tasks within an entire discipline, two things are required: a pretraining data set that contains a large amount 16 of training samples and is representative of the feature distribution studied across the discipline and a suitable pretraining task (Ruder, 2018, 2019). 4.3.1 Pretraining Tasks The most fundamental pretraining approaches in NLP are self-supervised (Ruder, 2019). Among these, a very common pretraining task is language modeling (Bengio et al., 2003). A language model learns to assign a probability to a sequence of tokens (Bengio et al., 2003). As the probability for a sequence of T tokens, P(a1, . . . , at , . . . , aT ), can be computed as P(a1, . . . , at , . . . , aT ) = Y T t=1 P(at |a1, . . . , at−1) (10) or as P(a1, . . . , at , . . . , aT ) = Y 1 t=T P(at |aT , . . . , at+1) (11) language modeling involves predicting the conditional probability of token at given all its preceding tokens, P(at |a1, . . . , at−1), or implicates predicting the conditional probability of token at given all its succeeding tokens, P(at |aT , . . . , at+1) (Bengio et al., 2003; Yang et al., 2020). A forward language model models the probability in equation 10, a backward language model computes the probability in equation 11 (Peters et al., 2018). When being trained on a forward and/or backward language modeling task in pretraining, a model is likely to learn general structures and aspects of language, such as long-range dependencies, compositional structures, semantics and sentiment, that are relevant for a wide range of possible target tasks (Howard and Ruder, 2018; Ruder, 2018). Hence, language modeling can be argued to be a well-suited pretraining task (Howard and Ruder, 2018). 4.3.2 Pretraining Data Sets In contrast to computer vision where pretraining on the ImageNet Dataset is ubiquitous, there is no standard pretraining data set in NLP (Aßenmacher and Heumann, 2020; Ruder, 2019). The text corpora that have been employed for pretraining vary widely regarding the number of tokens they contain as well as their accessibility (Aßenmacher and Heumann, 2020).6 Most models are trained on a combination of different corpora. Several models (e.g. Devlin et al., 2019; Lan et al., 2020; Liu et al., 2019; Yang et al., 2019) use the English Wikipedia and the BooksCorpus Dataset (Zhu et al., 2015). Many models (e.g. Brown et al., 2020; Liu et al., 2019; Radford et al., 2019; Yang et al., 2019) additionally also use pretraining corpora made up of web documents obtained from crawling the web. 6A detailed and systematic overview over these data sets is provided by Aßenmacher and Heumann (2020). 17 Note that whereas pretraining is not standardized, there are common benchmark data sets that are used when it comes to comparing the performance of different learning models on NLP target tasks. Widely utilized benchmark data sets, that together cover a large variety of NLP tasks, are the General Language Understanding Evaluation (GLUE) (Wang et al., 2019b), SuperGLUE (Wang et al., 2019a), the Stanford Question Answering Dataset (SQUAD) (Rajpurkar et al., 2016, 2018) and the ReAding Comprehension Dataset From Examinations (RACE) (Lai et al., 2017). For more information on GLUE, SuperGLUE, SQUAD and RACE see the explanatory box below. —————————————————————————————————— NLP Tasks. GLUE comprises nine English language understanding subtasks in which binary or multi-class classification problems have to be solved (Wang et al., 2019b). The tasks vary widely regarding training data set size, textual style, and difficulty (Wang et al., 2019b). The Stanford Sentiment Treebank (SST-2) (Socher et al., 2013) subtask, for example, is a single-sentence binary classification task in which the sentiments expressed in sentences from movie reviews are to be classified as being positive vs. negative (Wang et al., 2019b). In the subtask based on the Multi-Genre Natural Language Inference Corpus (MNLI) (Williams et al., 2018), in contrast, the model is presented with a sentence pair consisting of a premise and a hypothesis. The model has to make a prediction whether the premise entails, contradicts, or neither entails nor contradicts the hypothesis (Wang et al., 2019b). SuperGLUE was introduced when the performance of deep neural networks used for transfer learning surpassed human performance on GLUE (Wang et al., 2019a). SuperGLUE comprises eight subtasks considered to be more difficult than the GLUE tasks, e.g. natural language inference or reading comprehension tasks (Wang et al., 2019a). Models that are evaluated based on SQUAD are presented with passages from Wikipedia articles and corresponding questions (Rajpurkar et al., 2016). The task is to return the text segment answering the question or discern that the question is not answerable by the provided passage (Rajpurkar et al., 2016, 2018). Finally, RACE is made up of 97, 687 multiple choice questions on 27, 933 text passages taken from English exams for Chinese students testing reading comprehension ability (Lai et al., 2017). —————————————————————————————————— 4.3.3 Deep Pretraining Another issue beside a suitable pretraining task and a large pretraining corpus is the depth (i.e. the number of hidden layers) of the neural language representation model. From the perspective of transfer learning, the early seminal word embedding models, such as continuous bag-of-words (CBOW) (Mikolov et al., 2013a), skip-gram (Mikolov et al., 2013a,b), and Global Vectors (GloVe) (Pennington et al., 2014), are self-supervised 18 pretraining approaches (Ruder, 2019). In these models, word embeddings are learned based on neural network architectures that have no hidden layer (Ruder, 2019). This implies that these models for each term learn one vector representation, that encodes one information (Peters et al., 2018). Representing each term with a single vector, however, may become problematic if the meaning of a term changes with context (e.g. ‘party’) (Peters et al., 2018). Moreover, for models to deduce meaning from sequences of words, several different types of information (e.g. syntactic and semantic information) are likely to be required (Peters et al., 2018). In contrast to the early word embedding models, deep learning models learn multilayered contextualized representations (e.g. McCann et al., 2018; Howard and Ruder, 2018; Peters et al., 2018). In deep neural networks for NLP, each layer learns one vector representation for a term (Peters et al., 2018). Hence, a single term is represented by several vectors—one vector from each layer. Although it cannot be specified a priori which information is encoded in which hidden layer in a specific model trained on a specific task, one can expect that the information encoded in lower layers is less complex and more general whereas information encoded in higher layers is more complex and more task-specific (Yosinski et al., 2014). The representations learned by a deep neural language model thus may, for example, encode syntactic aspects at lower layers and semantic context-dependent information in higher layers (see for example Peters et al., 2018). As comparisons across NLP benchmark data sets show, deep neural language models provide contextualized representations of language that generalize better across a wide range of specific target tasks compared to the one layer representations from early word embedding architectures (see e.g. McCann et al., 2018). 4.4 Adaptation: Feature Extraction vs. Fine-Tuning There are two basic ways of how to implement the adaptation phase in transfer learning: feature extraction vs. fine-tuning (see Figure 5) (Ruder, 2019). In a feature extraction approach the representations learned in the pretraining phase are frozen and not altered during adaptation (Ruder, 2019). In fine-tuning, in contrast, the pretrained parameters are not fixed but are updated in the adaptation phase (Ruder, 2019). An example for a feature extraction approach is ELMo (Embeddings from Language Models) (Peters et al., 2018). In pretraining, ELMo learns three layers of word vectors (Peters et al., 2018). These learned representations then are frozen and extracted to serve as the input for a target task-specific model that learns a linear combination of the three layers of pretrained vectors (Peters et al., 2018). Here, only the weights of the linear model but not the frozen pretrained vectors are trained (Peters et al., 2018). In fine-tuning typically the same model architecture used in pretraining is also used for adaptation (Peters et al., 2019). The model is merely added a task-specific output layer (Peters et al., 2019). The parameters learned in the pretraining phase serve as initializations for the model in the adaptation phase (Ruder, 2019). When training the model on the target task, the gradients are allowed to backpropagate to the pretrained 19 ."#$%&" "'$&#($)*+ .)+" − $%+)+- .."#$%& .."#$%& .."#$%& .'($)&' .."#$%& ⇝ .'($)&' .."#$%& ⇝ .'($)&' .."#$%& ⇝ .'($)&' .."#$%& ⇝ .'($)&' Figure 5: Feature Extraction vs. Fine-Tuning in the Adaptation Phase. In a feature extraction approach the parameters learned during pretraining on the source task are not changed. Only the parameters in the separate target-task model architecture are trained. In fine-tuning, the entire pretrained model architecture is trained on the target task such that also the pretrained parameters are updated. representations and thus to induce changes on these pretrained representations (Ruder, 2019). In contrast to the feature extraction approach, the pretrained parameters hence are allowed to be fine-tuned to capture task-specific adjustments (Ruder, 2019). When fine-tuning BERT on a target task, for example, a target task-specific output layer is put on top of the pretraining architecture (Devlin et al., 2019). Then the entire architecture is trained, meaning that all parameters (also those learned in pretraining) are updated (Devlin et al., 2019). The performance of feature extraction vs. fine-tuning is likely to be a function of the dissimilarity between the source and the target task (Peters et al., 2019). At present, the highest performances in text classification and sentiment analysis tasks are achieved by pretrained language representation models that are fine-tuned to the target task (Ruder, 2020). Fine-tuning, however, can be a tricky matter. The central parameter in fine-tuning is the learning rate η with which the gradients are updated during training on the target task (see equation 8) (Ruder, 2019). Too much fine-tuning (i.e. a too high learning rate) can lead to catastrophic forgetting—a situation in which the general representations learned during pretraining are overwritten and therefore forgotten when fine-tuning the model (Kirkpatrick et al., 2017). A too careful fine-tuning scheme (i.e. a too low learning rate), in contrast, may lead to a very slow convergence process (Howard and Ruder, 2018). In general it is recommended that the learning rate should be lower than the learning rate used during pretraining such that the representations learned during pretraining are not altered too much (Ruder, 2019). 20 To effectively fine-tune a pretrained model without catastrophic forgetting, Howard and Ruder (2018) present a set of learning rate schedules that vary the learning rate over the course of the adaptation phase. They also introduce discriminative fine-tuning—a procedure in which each layer has a different learning rate during fine-tuning to account for the fact that different layers encode different types of information (Howard and Ruder, 2018; Yosinski et al., 2014). 5 Attention-Based Natural Language Processing: The Transformer As RNNs and derived architectures as the LSTM model are made to process sequential input data, they seem the natural model of choice for processing sequences of textual tokens. The problem of recurrent architectures, however, is that they model dependencies by sequentially propagating through the positions of the input sequence. Whether or how well dependencies are learned by RNNs is a function of the distance between the tokens that depend on each other (Goodfellow et al., 2016). The longer the distance between the tokens, the less well the dependency tends to be learned (Goodfellow et al., 2016). A solution to this problem is provided by the attention mechanism, first introduced by Bahdanau et al. (2015) for Neural Machine Translation (NMT), that allows to model dependencies between tokens irrespective of the distance between them. The Transformer is a deep learning architecture that is solely based on attention mechanisms (Vaswani et al., 2017). It overcomes the inefficiencies of recurrent models and introduces self-attention (Vaswani et al., 2017). To provide a solid understanding of these methods, this section first explains the attention mechanism and then introduces the Transformer. 5.1 The Attention Mechanism The common task encountered in NMT is to translate a sequence of S tokens in language G, {g1, . . . , gs, . . . , gS}, to a sequence of T tokens in language A, {a1, . . . , at , . . . , aT } (Sutskever et al., 2014). The length of the input and output sequences may differ: ‘He is giving a speech.’ translated to German is ‘Er h¨alt eine Rede.’. The task thus is one of sequence-to-sequence modeling whereby the sequences are of variable length (Sutskever et al., 2014). The classic architecture to solve this task is an encoder-decoder structure (see Figure 6) (Sutskever et al., 2014). The encoder maps the input tokens {g1, . . . , gs, . . . , gS} into a single vector of fixed dimensionality, context vector c, that is then provided to the decoder that generates the sequence of output tokens {a1, . . . , at , . . . , aT } from c (Sutskever et al., 2014). In the original NMT articles, encoder and decoder are recurrent models (Sutskever et al., 2014). Hence, the encoder sequentially processes each input token embedding z[gs] . The hidden state at time step s, hs, is a nonlinear function of the 21 ℎ. ℎ" ℎ. #[,.] #[,"] #[&.] … #[&"] … ℎ" #[,%] #[,&] #[,'] #[,(] ℎ( ℎ) ℎ* ℎ+ = % ℎ# … &'%()*+ ,*%()*+ #[,(] () *+ /*9*./ , +0)):ℎ [345] [345] 37 ℎä1- … Figure 6: Encoder-Decoder Architecture. Encoder-decoder structure in neural machine translation. In this example, the six token input sentence {He, is, giving, a, speech, [EOS]} is translated to German: {Er, h¨alt, eine, Rede, [EOS]}. The end-of-sentence symbol [EOS] is used to signal to the model the end of a sentence. The encoder processes one input token embedding z[gs] at a time and updates the input hidden state hs at each time step. The last encoder hidden state h6 serves as context vector c that captures all the information from the input senquence. The decoder generates one translated output token at a time. Each output hidden state ht is a function of the preceding hidden state ht−1, the preceding predicted output token embedding z[at−1] , and context vector c. 22 previous hidden state, hs−1, and input token embedding z[gs] (Cho et al., 2014): hs = σ(hs−1, z[gs] ) (12) The last encoder hidden state, hS, corresponds to context vector c that then is passed on to the decoder which—given the information encoded in c—produces a variable-length sequence output {a1, . . . , at , . . . , aT }. The decoder also operates in a recurrent manner: one output token at is produced at one time step. In contrast to the encoder, the hidden state of the decoder at time step t now is not only a function of the previous hidden state ht−1 but also the embedding of the predicted previous output token z[at−1] , and context vector c (see also Figure 6) (Cho et al., 2014): ht = σ(ht−1, z[at−1] , c) (13) A problem with this traditional encoder-decoder structure is that all the information about the input sequence—regardless of the length of the input sequence—is captured in a single context vector c (Bahdanau et al., 2015). The attention mechanism, that has been introduced to NMT by Bahdanau et al. (2015) and was refined by Luong et al. (2015), resolves this problem. In the attention mechanism, the encoder produces hidden states for each input token and at each time step the decoder can attend to, and thus derive information from, all encoder-produced input hidden states when computing its hidden state ht (see Figure 7). More precisely, the decoder hidden state at time point t, ht , is a function of the initial decoder hidden state h˜t , the previous output token z[at−1] , and an output token-specific context vector ct (Luong et al., 2015).7 ht = σ(h˜t , z[at−1] , ct) (14) Note that now at each time step there is a context vector ct that is specific to the tth output token (Bahdanau et al., 2015). The attention mechanism rests in the computation of ct , which is a weighted sum over the input hidden states {h1, . . . , hs, . . . , hS} (Bahdanau et al., 2015): ct = X S s=1 αtshs (15) The weight αts is computed as αts = exp(score(h˜t , hs)) PS∗ s ∗=1 exp(score(h˜t , hs ∗ )) (16) whereby score is a scoring function assessing the compatibility between output token representation h˜t and input token representation hs (Luong et al., 2015). score could 7Note that equation 14 blends the specifications of Luong et al. (2015) and Bahdanau et al. (2015). Luong et al. (2015) do not include z[at−1] . Luong et al. (2015) also do not explicitly state how they compute h˜t. Bahdanau et al. (2015) use ht−1 instead of h˜t to represent the state of the decoder at t (or rather: at the moment just before producing the tth output token). 23 be, for example, the dot product of h˜t and hs (Luong et al., 2015). The weight αts is a measure of the degree of alignment of the sth input token, represented by hs, with the tth output token, represented as h˜t . αts is the probability that output token h˜t is aligned with (in the context of NMT: translated from) input token hs (Bahdanau et al., 2015). ℎ. ℎ" ℎ. #[,.] #[,"] #[&.] … #[&"] … ℎ" #[,%] #[,&] #[,'] #[,(] ℎ( ℎ) ℎ* ℎ+ ℎ# … &'%()*+ ,*%()*+ #[,(] () *+ /*9*./ , +0)):ℎ [345] [345] 37 ℎä1- … ℎ.# %# '%. '%" '%$ '%# '%& '%' Figure 7: Attention in an Encoder-Decoder Architecture. Visualization of the attention mechanism in an encoder-decoder structure at time step t. In the attention mechanism, at each time step, i.e. for each output token, there is a token-specific context vector ct. ct is computed as the weighted sum over all input hidden states {h1, . . . , h6}. The weights are {αt,1, . . . , αt,6}. αt,1 captures the alignment between the tth output token, as represented by the initial output hidden state h˜t, and input token hidden state h1. Input hidden states that do not match with output token representation h˜t receive a small weight such that their contribution vanishes, whereas input hidden states that are relevant to output token h˜t receive high weights, thereby increasing their contribution (Alammar, 2018c). Hence, ct considers all input hidden states and especially attends to those input hidden states that match with the current output token. As context vector ct is constructed at each time step, i.e. for each output token, based on a weighted sum of all input hidden states, the attention architecture allows for modeling dependencies between tokens irrespective of their distance (Vaswani et al., 2017). 5.2 The Transformer The introduction of the attention mechanism constituted a significant development in sequence-to-sequence modeling. Yet, the original articles on attention use recurrent architectures in the encoder and decoder. The sequential nature of recurrent models implies that within each training example sequence each token has to be processed one after another—a computationally not efficient strategy (Vaswani et al., 2017). To overcome this inefficiency and to enable parallel processing within training sequences, 24 Vaswani et al. (2017) introduced the Transformer architecture that completely abandons recurrence and solely rests on attention mechanisms. The Transformer consists of a sequence of six encoders followed by a stack of six decoders (see Figure 8) (Vaswani et al., 2017).8 Each encoder consists of two components: a multi-head self-attention layer (to be explained below) and a feedforward neural network (Vaswani et al., 2017). Each decoder also has a multi-head self-attention layer followed by a multi-head encoder-decoder attention layer and a feedforward neural network (Vaswani et al., 2017). In the following, the structure and workings of the Transformer are explicated in detail. The Transformer architecture is depicted in Figure 8 and a comprehensive visualization of the precisely explicated Transformer encoder is provided in Figure 9. When reading through this outline, two aspects are to be kept in mind. First, instead of processing each token embedding of each training example one after another, the Transformer encoder takes as an input the whole set of embeddings for one training example and processes this set of embeddings in parallel (Alammar, 2018b). Second, the focus of the following explanation will be on the first of the Transformer encoders. The other encoders operate in the same way with the exception that they do not take as an input a set of T word embeddings, {z[a1] , . . . , z[at] , . . . , z[aT ]}, but T updated vector representations, denoted as {h ∗ 1 , . . . , h ∗ t , . . . , h ∗ T }, that are produced by the previous encoder (see upper part of Figure 9) (Alammar, 2018b). Moreover, the T word embeddings entering the first encoder are position-aware word embeddings (see bottom of Figure 9) (Vaswani et al., 2017). A position-aware word embedding is the sum of a pure word embedding vector and a positional encoding vector (Vaswani et al., 2017). The positional encoding vector contains information on the position of the tth token within the input sequence, thereby making the model aware of token positions (Alammar, 2018b). 5.2.1 Multi-Head Self-Attention Layer The first element in a Transformer encoder is the multi-head self-attention layer. In the self-attention layer, the provided input sequence attends to itself. Instead of improving the representation of an output token by attending to relevant tokens in the input sequence, the idea of self-attention is to improve the representation of an input token by attending to the tokens in the sequence in which it is embedded in (Alammar, 2018b). For example: If ‘The company is issuing a statement as it is bankrupt.’ were a sentence to be processed, then the word embedding for the word ‘it’ that enters the Transformer would not contain any information regarding which other token in the sentence ‘it’ is referring to. Is it the company or the statement. In the self-attention mechanism the representation for ‘it’ is updated by attending to, and incorporating information from, other relevant tokens in this sentence (Alammar, 2018b). It therefore is to be 8Note that the number of encoders and decoders as well as the dimensionality of the input word embeddings and the key, query and value vectors (introduced in the following) are Transformer hyperparameters that are simply set by the authors to specific values. Other suitable values could be used instead. 25 (,<.@%) 1 B%<.@%) 1 B%<.@%) 2 B%<.@%) 3 B%<.@%) 4 B%<.@%) 5 B%<.@%) 6 (,<.@%) 2 (,<.@%) 3 (,<.@%) 4 (,<.@%) 5 (,<.@%) 6 H[*.] H[*#] H[*"] H[*$] H[*%] H[*&] ,. -. ,# -# ,$ -$ ," -" ,% -% ,& -& I&,%+) +,@ 2.J*K+L M).;+;&/&*> B&'*)&;=*&., .7%) N.<+;=/+)> Figure 8: Transformer Architecture. In the original article by Vaswani et al. (2017), the Transformer is made up of a stack of six encoders proceeded by a stack of six decoders. In contrast to recurrent architectures where each input token is handled one after another, the Transformer encoders processes the entire set of input token representations in parallel. Here the input token embeddings are {z[a1] , . . . , z[a6]}. The sixth encoder passes the key and query vectors of the input tokens, {k1, q1, . . . , k6, q6}, to each of the decoders. The decoders generate a prediction for one output token at a time. The hidden state of the last decoder is handed to a linear and softmax layer to produce a probability distribution over the vocabulary signifying the prediction for the next output token. 26 .[-#] .[-"] .[-.] .[-$] .[-%] .[-&] ./0. ./0# ./0$ ./0" ./0% ./0& .[-#] ∗ 12 .[-"] ∗ .[-.] ∗ .[-$] ∗ .[-%] ∗ .[-&] ∗ 30 73>367 4 0.22.ℎ 9:; + + + + + + = = = = = = ,. -. >. ,# -# ># ,$ -$ >$ ," -" >" ,% -% >% ,& -& >& /. /" /$ /# /& /' 0. 0" 0$ 0# 0& 0' 0. ∗ 0" ∗ 0$ ∗ 0# ∗ 0& ∗ 0' ∗ ℎ. ℎ" ℎ$ ℎ# ℎ& ℎ' ℎ. ∗ ℎ" ∗ ℎ$ ∗ ℎ# ∗ ℎ& ∗ ℎ' ∗ '()* +,-.**/012 3(2/4/(056 +,-.**/012 3(2/4/(0-785). +,-.**/012 9:64/-;.5* <.6=-744.04/(0 >5..) @..*=()85)* A.:)56 A.48()B (,<.A%) 1 C.2/*:56 >.5)0- /01 & >5..) A(),56/E54/(0 C.2/*:56 >.5)0- /01 & >5..) A(),56/E54/(0 ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... Figure 9: Transformer Encoder Architecture. This visualization details the processes in the first Transformer encoder. The encoder comprises a multi-head self-attention layer and a feedforward neural network (FNN); each followed by residual learning and layer normalization. The first encoder takes as an input position-aware word embeddings, {z[a1] , . . . , z[a6]}, that are transformed into eight sets of key, query and value vectors. One set is {k1, q1, v1, . . . , k6, q6, v6}. These are processed in the multi-head self-attention layer to produce eight sets of context vectors (one set being {c1, . . . , c6}). The sets then are concatenated and transformed linearly to become the updated representations {u1, . . . ,u6}. After residual learning and layer normalization, {u ∗ 1 , . . . ,u ∗ 6 } enter the FNN, whose output—after residual learning and layer normalization— are the updated representations produced by the first Transformer encoder: {h ∗ 1 , . . . , h ∗ 6 }. The representations {h ∗ 1 , . . . , h ∗ 6 } constitute the input to the next encoder, where they are first transformed to sets of key, query and value vectors.27 expected that after passing through the stack of encoders with self-attention layers, the representation of ‘it’ absorbed some of the representation for ‘company’ and so encodes information on the alignment between ‘it’ and ‘company’ (Alammar, 2018b). The first operation within a self-attention layer is that each input word embedding z[at] is transformed into three separate vectors, called key kt , query qt , and value vt (see Figure 9). The key, query, and value vectors are three different projections of the input token embedding z[at] (Alammar, 2018b). They are generated by matrix multiplication of z[at] with three different weight matrices, Wk, Wq, and Wv (Vaswani et al., 2017):9 kt = z[at]Wk qt = z[at]Wq vt = z[at]Wv (17) Then, for each token t context vector ct is computed as a weighted sum of the value vectors of all tokens {1, . . . , t∗ , . . . , T∗} that are in the same sequence as t (Vaswani et al., 2017): ct = T X∗ t ∗=1 αt,t∗ vt ∗ (18) The weight αt,t∗ captures the probability that token t, represented by qt , is aligned with token t ∗ that is represented as kt ∗ : αt,t∗ = exp(score(qt , kt ∗ )) PT ∗ t ∗=1 exp(score(qt , kt ∗ )) (19) whereby score is (qtk > t ∗ )/ p |kt ∗ | (Vaswani et al., 2017). Thus attention vector ct is calculated as in a basic attention mechanism—except that the attention now is with respect to the value vectors of the tokens t ∗ that are in the same sequence as token t (see also Figure 10).10 The self-attention mechanism outlined so far is conducted eight times in parallel (Vaswani et al., 2017). Hence, for each token t eight different sets of query, key and value vectors are generated and there will be not one but eight attention vectors {ct1, . . . , ct8} (Vaswani et al., 2017). In doing so, each attention vector can attend to different tokens in each of the eight different representation spaces (Vaswani et al., 2017). For example, in one representation space the attention vector for token t may learn syntactic structures and in another representation space the attention vector may attend to semantic connections (Vaswani et al., 2017). In the example sentence from above, the first attention 9Note that for reasons of computational efficiency, kt, qt, and vt typically are set to have a lower dimensionality than z[at] (Vaswani et al., 2017). In the original article Vaswani et al. (2017) let the word embeddings and the updated word vector representations outputted by the encoders have dimensionality 512, whereas each of kt, qt, and vt has dimensionality 64. 10Note that in a self-attention mechanism the query vector of token t, qt, is scored with the key vectors of all tokens in the same sequence—and thus also with itself (see Figure 10). The score of the query vector of t, qt, and the key vector of t, kt, is likely to be very high compared to the other scores (Alammar, 2018b). Yet, the model also is likely to learn that attention to other tokens in the sequence can be beneficial for representing token t (Alammar, 2018b). 28 (. <[,.] <[,"] <[,#] <[,$] <[,%] <[,..] %- '(,. ). *. <[,&] <[,'] <[,(] <[,)] <[,.*] (" )" *" ($ )$ *$ (# )# *# (& )& *& (' )' *' (* )* ** (( )( *( (+ )+ *+ (.,).,*., (..)..*.. '(," '(,$ '(,# '(,& '(,' '(,* '(,( '(,+ '(,., '(,.. 9ℎ% <..@.+,> &' &''=&,- + '*+*%@%,* +' &* &' ;+,5)=.* [(12] Figure 10: Attention Mechanism in the Transformer. Illustration of the attention mechanism in the first Transformer encoder for the 8th token (‘it’) in the example sentence ‘The company is issuing a statement as it is bankrupt.’. The arrows pointing from the value vectors {v1, . . . , v11} to context vector c8 are the weights {α8,1, . . . , α8,t∗ , . . . , α8,11}. A single weight α8,t∗ gives the probability that token 8, represented by q8, is aligned with token t ∗ represented as kt ∗ . The larger α8,t∗ is assumed to be in this example, the thicker the arrow and the darker the corresponding value vector. The dotted lines symbolize the computation of the weights {α8,1, . . . , α8,t∗ , . . . , α8,11}. 29 vector for the token ‘it’, c8,1, may have a high attention weight for ‘company’, whereas another attention vector, say c8,3, may more strongly attend to ‘bankrupt’ (Alammar, 2018b). Because of the self-attention mechanism is implemented eight times in parallel and generates eight attention vectors (or heads), the procedure is called multi-head self-attention (Vaswani et al., 2017). The eight attention vectors subsequently are concatenated into a single vector, ct = [ct1; . . . ; ct8], and multiplied with a corresponding weight matrix W0 to produce vector ut (Vaswani et al., 2017). ut constitutes an updated representation of token t. It incorporates the information of tokens in the same sequence that was captured by selfattention. Before being passed to the feedforward neural network, ut is added to z[at] , thereby allowing for residual learning (He et al., 2015).11 Then, layer normalization as suggested in Ba et al. (2016) is conducted to reduce training time (Vaswani et al., 2017).12 u ∗ t = LayerNorm(ut + z[at] ) (20) 5.2.2 Feedforward Neural Network u ∗ t then enters the feedforward neural network with a ReLU activation function (Vaswani et al., 2017) ht = max(0,u ∗ t W1 + b1)W2 + b2 (21) followed by a residual connection with layer normalization (Vaswani et al., 2017): h ∗ t = LayerNorm(ht + u ∗ t ) (22) h ∗ t finally is the representation of token t produced by the encoder. The entire sequence of representations, {h ∗ 1 , . . . , h ∗ t , . . . , h ∗ T }, serves as the input for the next encoder that generates eight sets of query, key and value vectors from each representation h ∗ t to implement multi-head self attention and to finally produce an updated set of representations, {h ∗ 1 , . . . , h ∗ t , . . . , h ∗ T } ∗ , that are passed to the next encoder and so on. The last encoder from the stack of encoders produces the key and value vectors from its produced sequence of updated representations and passes these to each encoder-decoder multi-head attention layer in each decoder (see again Figure 8) (Vaswani et al., 2017). The decoder then generates one output token t at a time (Vaswani et al., 2017). The 11In residual learning, instead of leaning a new representation in each layer, merely the residual change is learned (He et al., 2015). Here ut can be conceived of as the residual on the original representation z[at] . Residual learning has been shown to facilitate the optimization of very deep neural networks (He et al., 2015). 12In layer normalization, for each training instance, the values of the hidden units within a layer are standardized by using the mean and standard deviation of the layer’s hidden units (Ba et al., 2016). Layer normalization reduces training time and enhances generalization performance due to its regularizing effects (Ba et al., 2016). 30 output of the last decoder is processed through a linear layer and a softmax layer to produce the Transformer output, which is a probability distribution over the terms in the vocabulary predicting the next output token (Vaswani et al., 2017). Except for the encoder-decoder attention layer in which the decoder pays attention to the encoder input, the architecture of each decoder is largely the same as those of the encoders. One important difference, however, is that whereas each encoder is provided with the hidden representations of the entire input sequence, each decoder receives as an input the hidden representations of only those tokens the stack of decoders has predicted as an output so far (Alammar, 2018b). Consequently, the multi-head self-attention layer of each decoder is masked meaning that the attention vector for output token t can only attend to output tokens preceding token t (Vaswani et al., 2017). 6 Transfer Learning with Transformer-Based Models The Transformer advances the study of text as it enables the representation for each token to encode information from other tokens in the same sequence. Irrespective of the distance between the tokens, this allows for modeling dependencies between tokens and context-dependent meanings of tokens. The utilization of pretrained representations in a transfer learning setting is likely to increase prediction performances in text-based supervised learning tasks—especially for small training corpora. Taken together, the Transformer architecture in combination with transfer learning literally transformed the field of NLP. After the introduction of the Transformer by Vaswani et al. (2017) several models for transfer learning that included elements of the Transformer were developed (e.g. Clark et al., 2020; Devlin et al., 2019; Lan et al., 2020; Liu et al., 2019; Radford et al., 2018; Raffel et al., 2020; Yang et al., 2019). These models and their derivatives significantly outperformed previous state-of-the-art models and so far generate the highest performances on a wide spectrum of NLP tasks (see for example the leaderbords of common benchmark datasets as GLUE13, SuperGLUE14, SQuAD15 and RACE16). An important step within these developments was the introduction of BERT (Devlin et al., 2019). By establishing new state-of-the-art performance levels for eleven NLP tasks (Devlin et al., 2019), BERT demonstrated the power of transfer learning and caused great excitement in the NLP community (Alammar, 2018a). The introduction of BERT finally paved the way to a new transfer learning-based mode of learning in which it is common to use a pretrained language representation model off-the-shelf and adapt it to a specific target task as needed (Alammar, 2018a). BERT can be considered the seminal Transformer-based model for transfer learning. 13https://gluebenchmark.com/leaderboard 14https://super.gluebenchmark.com/leaderboard 15https://rajpurkar.github.io/SQuAD-explorer/ 16http://www.qizhexie.com/data/RACE leaderboard.html 31 BERT is a suitable starting point for social scientists that seek to explore and leverage the benefits of deep learning, transfer learning and the Transformer architecture. Yet, BERT is just one step of the continuing developments within NLP. Simultaneously with and independently of BERT as well as later as a response to or an extension of BERT a wide spectrum of Transformer-based models for transfer learning have been developed. This section first introduces BERT and then provides an overview over further models such that users know which (type of) Transfomer-based pretrained language representation model addresses which issue and thus is likely to be suitable for their target task. 6.1 BERT The classic language modeling pretraining task (see equations 10 and 11), that for example is employed by language representation models such as ELMo, ULMFiT and OpenAI GPT, has one disadvantage: it is strictly unidirectional (Devlin et al., 2019). A forward language model predicts the probability for the next token t given the so far predicted tokens, P(at |a1, . . . , at−1). Here, the model can only access information from the preceding tokens {a1, . . . , at−1} but not from the following tokens {at+1, . . . , aT }. In a self-attention mechanism this means that the context vector for token t can merely attend to, and hence can only incorporate information from, the representations of preceding but not from succeeding tokens (Devlin et al., 2019). The same is true for a backward language model in which the next token is predicted given all its following tokens, P(at |at+1, . . . , aT ) (Yang et al., 2019). A backward language model can only operate on, and capture information from, succeeding tokens. There is reason to believe that a representation of token t from a bidirectional model that simultaneously can attend to preceding and succeeding tokens constitutes a better representation of token t than a representation stemming from a unidirectional language model (Devlin et al., 2019). The concatenation of representations learned by a forward language model with the representations of a backward language model, however, does not generate representations that genuinely draw from left and right contexts (Devlin et al., 2019). The reason is that the forward and backward representations are learned separately and each representation captures information only from a unidirectional context (Yang et al., 2019). The authors inventing BERT sought to tackle this issue: BERT is a language representation model for sequential transfer learning that utilizes the Transformer encoder and masked language modeling, which is an adapted variant of the traditional language modeling pretraining task, to learn deep and bidirectional representations (Devlin et al., 2019). The following subsections provide information on the basic architecture of BERT, explain the masked language modeling task, explicate the input format required by BERT, and describe the pretraining and fine-tuning specifications. Figures 11 and 12 illustrate these aspects. 32 6.1.1 Architecture BERT consists of a stack of Transformer encoders17 and comes in two different model sizes (Devlin et al., 2019): BERTBASE consists of 12 stacked Transformer encoders. In each encoder there are 12 attention heads in the multi-head self-attention layer. The dimensionality of the input word embeddings and the updated hidden word vector representations is 768. With this model size, BERTBASE has 110 million parameters. BERTLARGE has 24 Transformer encoders with 16 attention heads and a hidden vector size of 1024. BERTLARGE has 340 million parameters. As in the original Transformer, the first BERT encoder takes as an input a sequence of embedded tokens, {z[a1] , . . . , z[at] , . . . , z[aT ]}, processes the embeddings in parallel through the self-attention layer and the feedforward neural network to generate a set of updated token representations, {h ∗ 1 , . . . , h ∗ t , . . . , h ∗ T }, that are then passed to the next encoder that also generates updated representations to be passed to the next encoder and so on until the representations finally enter output layers for prediction (Alammar, 2018a)."""
input_tokenized = tokenizer.encode(text, return_tensors='pt', max_length=16384,truncation=True).to(device)

dim = list(input_tokenized.size())
print("Input Shape (before padding to max_length): {}".format(dim))


input_tokenized = tokenizer.encode(text, return_tensors='pt',padding=padding,pad_to_max_length=True, max_length=16384,truncation=True).to(device)

dim = list(input_tokenized.size())
print("Input Shape: {} (padded to max_length)".format(dim))

model = model.to(device)
summary_ids = model.generate(input_tokenized,
                                  num_beams=4,
                                  no_repeat_ngram_size=3,
                                  length_penalty=1.2,
                                  min_length=3500,
                                  max_length=5000)

dim = list(summary_ids.size())
print("Summary Shape: ", dim)
  
summ = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids][0]

print("Text: ")
print(text)
print("Summary: ")
print(summ)